# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 本問は **仕様で降順指定あり**（`ORDER BY rating DESC`）
* `NOT IN` 回避（本問では不要）
* 判定は **ID 基準**、表示は仕様どおり

## 1) 問題

* `映画テーブルから、ID が奇数かつ description が 'boring' ではない映画を抽出し、rating の降順で返せ。`
* 入力: `Cinema(id int, movie text/varchar, description text/varchar, rating numeric/real)`
* 出力: `id, movie, description, rating` を **rating 降順**で返す（同率は id で安定化）

## 2) 最適解（単一クエリ）

> 本問はウィンドウ不要。PostgreSQL では単純な条件抽出＋降順ソートが最速・明快です。

```sql
SELECT
  id,
  movie,
  description,
  rating
FROM cinema
WHERE (id % 2) = 1           -- 奇数ID
  AND description IS NOT NULL
  AND description <> 'boring' -- 'boring' を除外（NULL は含めない）
ORDER BY rating DESC, id DESC; -- 同率タイを id で安定化

Runtime 173 ms
Beats 86.40%

```

### 備考（等価な書き方）

* ビット演算でも可：`(id & 1) = 1`
* 大文字小文字を厳密一致に固定したい場合は `COLLATE "C"`（英数のみなら高速）
  `AND description COLLATE "C" <> 'boring'`

## 3) 要点解説

* **奇数判定**: `id % 2 = 1` でも `(id & 1) = 1` でも同等。整数 PK ならどちらも安全。
* **NULL の扱い**: 仕様解釈を明確化するため `IS NOT NULL` を明示（NULL を含めたいなら `OR description IS NULL`）。
* **順序の安定化**: 出力の再現性を上げるため `ORDER BY rating DESC, id DESC` を推奨。
* **識別子の大文字**: PostgreSQL は未引用識別子を小文字化するため、テーブル作成時に `"Cinema"` と引用していない限り `FROM cinema` が正。作成時に引用しているなら `FROM "Cinema"` に合わせる。

## 4) 計算量（概算）

* フルスキャン前提で **O(N)**。`ORDER BY rating DESC` でソートが発生する場合 **O(N log N)**。
* 適切な索引で I/O を減らせる（下記）。

### 実運用で効くインデックス（参考）

> 本問のような “条件でざっくり絞って rating で降順” は **式インデックス** と **降順インデックス** が効きます。

```sql
-- 1) 偶奇＋並び替え最適化（式インデックス）
CREATE INDEX IF NOT EXISTS idx_cinema_odd_rating_desc
ON cinema ( (id & 1), rating DESC, id DESC );

-- 2) 'boring' 除外を事前に省く（部分インデックス）
CREATE INDEX IF NOT EXISTS idx_cinema_not_boring_rating_desc
ON cinema (rating DESC, id DESC)
WHERE description IS NOT NULL AND description <> 'boring';
```

*データ分布の目安*

* “boring” が少数派 ⇒ ②（部分インデックス）が特に効く
* 全体から半分を取る偶奇判定 ⇒ ①でスキャン範囲を半減しつつ並び替えもカバー

`EXPLAIN (ANALYZE, BUFFERS)` で実測し、使用されているインデックスとソート有無（`Sort Method` や `Index Only Scan`）を確認するのがベストです。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Cinema] --> B[条件 id 奇数]
  B --> C[条件 description NOT NULL かつ <> boring]
  C --> D[rating 降順 かつ id で安定化]
  D --> E[出力 id, movie, description, rating]
```

この規模の単純フィルタ＋降順ソートで、**クエリ文だけ**ではほぼ頭打ちです。さらに詰めるなら “物理設計と実行計画” に踏み込みます。

---

## 伸びしろ（実運用向け）

### 1) 部分＋式＋降順の “カバリング” インデックス

対象行を事前に削り、`ORDER BY` も満たし、**Index Only Scan** を狙います。

```sql
-- 奇数かつ boring ではない行だけを対象に、並び替え順で貼る
CREATE INDEX IF NOT EXISTS idx_cinema_hot
ON cinema (rating DESC, id DESC) INCLUDE (movie, description)
WHERE (id & 1) = 1
  AND description IS NOT NULL
  AND description <> 'boring';
```

* `WHERE` で **走査対象そのものを縮小**
* `rating DESC, id DESC` で **ソート不要**（インデックス順に取り出し）
* `INCLUDE (movie, description)` で **Index Only Scan** 可（可視性マップが効けば）

> 既存クエリはそのままで OK（書き換え不要）。

### 2) 可視性マップを整えて Index Only を本物に

`VACUUM (ANALYZE)` を回し、更新が多い場合は定期運用に。
可視性マップが立たないと **Index Only** が **Index Scan + heap hit** に退化します。

```sql
VACUUM (ANALYZE) cinema;
-- 直近で大量更新があるなら autovacuum の閾値調整も検討
```

### 3) 統計＆並列のチューニング（軽め）

* `ANALYZE cinema;`（新インデックス作成後は必ず）
* 並列が効く環境なら
  `max_parallel_workers_per_gather` を 2〜4 に。
  小規模だと並列コストで逆効果なので **EXPLAIN (ANALYZE, BUFFERS)** で要確認。

### 4) 物理配置でキャッシュヒット率を上げる（任意）

* たまに `CLUSTER cinema USING idx_cinema_hot;`（ダウンタイム要）
  もしくは `pg_repack`。順序アクセスが連続化し I/O が滑らかになります。

---

## 実行計画の確認ポイント

```sql
EXPLAIN (ANALYZE, BUFFERS)
SELECT id, movie, description, rating
FROM cinema
WHERE (id % 2) = 1
  AND description IS NOT NULL
  AND description <> 'boring'
ORDER BY rating DESC, id DESC;
```

見るべき点：

* `Index Only Scan using idx_cinema_hot` になっているか
* `Sort` ノードが **無い**（＝インデックス順で取得）
* `Heap Fetches: 0` 付近（可視性マップが効いている）
* `Buffers: shared hit` が大半（キャッシュヒットが高い）

---

## クエリ本文（最終形）

```sql
SELECT
  id,
  movie,
  description,
  rating
FROM cinema
WHERE (id % 2) = 1
  AND description IS NOT NULL
  AND description <> 'boring'
ORDER BY rating DESC, id DESC;

Runtime 175 ms
Beats 79.80%

```

> 文面はそのまま、**インデックス設計と運用で短縮**する、が最短コースです。
> 目安として、部分インデックス＋IOS が噛めば **二桁ms** 台も十分に射程です（データ量・I/O 事情によります）。

